In [ ]:
import pygame
import random
import sys
import json

# Загрузка конфигурации из файла
with open('config.json', 'r') as f:
    config = json.load(f)

# Цвета из конфига
BG_COLOR = tuple(config["background_color"])
CURR_COLOR = tuple(config["current_cell_color"])
VISITED_COLOR = tuple(config["visited_cell_color"])
DEFAULT_COLOR = tuple(config["default_cell_color"])
WALL_COLOR = tuple(config["wall_color"])

# Настройки окна и лабиринта
WIDTH, HEIGHT = config["window_width"], config["window_height"]
CELL_SIZE = config["cell_size"]
MAZE_WIDTH, MAZE_HEIGHT = config["maze_width"], config["maze_height"]
FPS = config["animation_speed"]

# Инициализация Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Генератор Лабиринтов")
clock = pygame.time.Clock()

class Cell:       # клетка лабиринта
    def __init__(self, x, y):
        self.x = x     # Позиция в сетке
        self.y = y
        self.walls = {'top': True, 'right': True, 'bottom': True, 'left': True}
        self.visited = False     # Для алгоритма генерации
        self.current = False
         # Визуализация
    def draw(self):
        x = self.x * CELL_SIZE
        y = self.y * CELL_SIZE
        
        # Выбираем цвет клетки
        if self.current:
            color = CURR_COLOR
        elif self.visited:
            color = VISITED_COLOR
        else:
            color = DEFAULT_COLOR
        
        pygame.draw.rect(screen, color, (x, y, CELL_SIZE, CELL_SIZE))
        
        # Рисуем стенки
        if self.walls['top']:
            pygame.draw.line(screen, WALL_COLOR, (x, y), (x + CELL_SIZE, y), 2)
        if self.walls['right']:
            pygame.draw.line(screen, WALL_COLOR, (x + CELL_SIZE, y), (x + CELL_SIZE, y + CELL_SIZE), 2)
        if self.walls['bottom']:
            pygame.draw.line(screen, WALL_COLOR, (x, y + CELL_SIZE), (x + CELL_SIZE, y + CELL_SIZE), 2)
        if self.walls['left']:
            pygame.draw.line(screen, WALL_COLOR, (x, y), (x, y + CELL_SIZE), 2)
    # логика лабиринта
class Maze:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[Cell(x, y) for y in range(height)] for x in range(width)]
        self.stack = []
        self.current_cell = self.grid[0][0]
        self.current_cell.visited = True
        self.current_cell.current = True
        self.generation_complete = False
    
    def get_unvisited_neighbors(self, cell):
        neighbors = []
        directions = [('left', -1, 0), ('right', 1, 0), ('top', 0, -1), ('bottom', 0, 1)]
        for direction, dx, dy in directions:
            nx, ny = cell.x + dx, cell.y + dy
            if 0 <= nx < self.width and 0 <= ny < self.height:
                neighbor = self.grid[nx][ny]
                if not neighbor.visited:
                    neighbors.append((direction, neighbor))
        return neighbors
    
    def remove_wall(self, current, next_cell, direction):
        if direction == 'left':
            current.walls['left'] = False
            next_cell.walls['right'] = False
        elif direction == 'right':
            current.walls['right'] = False
            next_cell.walls['left'] = False
        elif direction == 'top':
            current.walls['top'] = False
            next_cell.walls['bottom'] = False
        elif direction == 'bottom':
            current.walls['bottom'] = False
            next_cell.walls['top'] = False
    # Алгоритм генерации (Backtracking)
    def generate_step(self):
        if self.generation_complete:
            return
        
        neighbors = self.get_unvisited_neighbors(self.current_cell)
        if neighbors:
            self.stack.append(self.current_cell)
            direction, next_cell = random.choice(neighbors)
            self.remove_wall(self.current_cell, next_cell, direction)
            self.current_cell.current = False
            next_cell.visited = True
            next_cell.current = True
            self.current_cell = next_cell
        elif self.stack:
            self.current_cell.current = False
            self.current_cell = self.stack.pop()
            self.current_cell.current = True
        else:
            self.current_cell.current = False
            self.generation_complete = True
    
    def draw(self):
        for row in self.grid:
            for cell in row:
                cell.draw()
    
    def export_to_file(self, filename="maze_export.txt"):
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(" Maze Generator Export\n")
            f.write(f"📏 Size: {self.width}x{self.height}\n")
            f.write("Legend: 🧱 - wall, 🟢 - path, 🚪 - entrance/exit\n\n")
        
        # Верхняя граница
            f.write('🧱' * (self.width * 2 + 1) + '\n')
        
            for y in range(self.height):
            # Вертикальные стенки
                line = '🧱'
                for x in range(self.width):
                    cell = self.grid[x][y]
                    line += '🟢'  # проход
                    if cell.walls['right']:
                        line += '🧱'
                    else:
                        line += '🟢'
                f.write(line + '\n')
            
            # Горизонтальные стенки
                line = '🧱'
                for x in range(self.width):
                    cell = self.grid[x][y]
                    if cell.walls['bottom']:
                        line += '🧱🧱'
                    else:
                        line += '🟢🧱'
                f.write(line + '\n')
        
        # Отмечаем вход и выход
            f.write("\n🚪 Entrance: top-left")
            f.write("\n🚪 Exit: bottom-right\n")

class Application:
    def __init__(self):
        self.maze = Maze(MAZE_WIDTH, MAZE_HEIGHT)
        self.paused = False
    
    def run(self):
        running = True
        font = pygame.font.SysFont(None, 24)
        
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_r:
                        # Сброс лабиринта
                        self.maze = Maze(MAZE_WIDTH, MAZE_HEIGHT)
                        self.paused = False
                    elif event.key == pygame.K_SPACE:
                        # Пауза/продолжение генерации
                        self.paused = not self.paused
                    elif event.key == pygame.K_e and self.maze.generation_complete:
                        self.maze.export_to_file("maze.txt")
                        print("Лабиринт экспортирован в maze.txt")
            
            if not self.paused and not self.maze.generation_complete:
                self.maze.generate_step()
            
            screen.fill(BG_COLOR)
            self.maze.draw()

            # Отображение инструкций
            screen.blit(font.render("R - Сброс лабиринта", True, WALL_COLOR), (10, HEIGHT - 75))
            screen.blit(font.render("SPACE - Пауза/Старт", True, WALL_COLOR), (10, HEIGHT - 50))
            screen.blit(font.render("E - Экспорт после завершения", True, WALL_COLOR), (10, HEIGHT - 25))
            
            pygame.display.flip()
            clock.tick(FPS)

        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    app = Application()
    app.run()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Лабиринт экспортирован в maze.txt
